In [1]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam, LBFGS, SGD
from torch.utils.data import Dataset, DataLoader

In [14]:
class PolynomialModel(nn.Module):
    def __init__(self, degree):
        super().__init__()
        self._degree = degree
        self.linear = nn.Linear(self._degree, 1)

    def forward(self, x):
        return self.linear(self._polynomial_features(x))

    def _polynomial_features(self, x):
        x = x.unsqueeze(1)
        return torch.cat([x ** i for i in range(1, self._degree + 1)], 1)

In [3]:
PolynomialModel(2)

PolynomialModel(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [4]:
import pandas as pd
import numpy as np
df1=pd.read_excel('Quiz_data_excel_sheet_2022.csv.xlsx')
df_inputs=df1.drop(['Y'],axis=1)
inputs_temp=df_inputs.to_numpy(dtype='float32')
#print(inputs)

df_op=df1['Y']
op_temp=df_op.to_numpy(dtype='float32')
#print(op.shape)
op_temp=np.reshape(op_temp,(-1,1))
inputs = torch.from_numpy(inputs_temp)
op = torch.from_numpy(op_temp)

In [5]:
def train_step(model, data, optimizer, criterion):
    running_loss = 0.0

    for i, (x, y) in enumerate(data):

        x_ = Variable(x, requires_grad=True)
        y_ = Variable(y.unsqueeze(1)) # unsqueeze to match dimensions with y_pred later

        def closure():
            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(x_)

            # Compute loss
            loss = criterion(y_pred, y_)

            # Backward pass
            loss.backward()

            return loss

        # Update weights
        optimizer.step(closure)

        # Update the running loss
        loss = closure()
        running_loss += loss.item()
    return running_loss

In [6]:
class DummyData(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [7]:
cubic_data = DummyData(inputs,op)
criterion = nn.MSELoss()


In [8]:
len(cubic_data)

1060

In [15]:
import torch.optim as optim
thirddeg = PolynomialModel(degree=3)
optimizer = optim.Adam(thirddeg.parameters(), lr=0.0001)


for epoch in range(250):
    running_loss = train_step(model=thirddeg,
                              data=cubic_data,
                              optimizer=optimizer,
                              criterion=criterion)
    print(f"Epoch: {epoch + 1:02}/250 Loss: {running_loss:.5e}")

Epoch: 01/250 Loss: 1.68960e+06
Epoch: 02/250 Loss: 1.63484e+06
Epoch: 03/250 Loss: 1.58075e+06
Epoch: 04/250 Loss: 1.52770e+06
Epoch: 05/250 Loss: 1.47565e+06
Epoch: 06/250 Loss: 1.42458e+06
Epoch: 07/250 Loss: 1.37447e+06
Epoch: 08/250 Loss: 1.32533e+06
Epoch: 09/250 Loss: 1.27713e+06
Epoch: 10/250 Loss: 1.22989e+06
Epoch: 11/250 Loss: 1.18359e+06
Epoch: 12/250 Loss: 1.13824e+06
Epoch: 13/250 Loss: 1.09384e+06
Epoch: 14/250 Loss: 1.05038e+06
Epoch: 15/250 Loss: 1.00787e+06
Epoch: 16/250 Loss: 9.66295e+05
Epoch: 17/250 Loss: 9.25663e+05
Epoch: 18/250 Loss: 8.85970e+05
Epoch: 19/250 Loss: 8.47215e+05
Epoch: 20/250 Loss: 8.09395e+05
Epoch: 21/250 Loss: 7.72511e+05
Epoch: 22/250 Loss: 7.36559e+05
Epoch: 23/250 Loss: 7.01536e+05
Epoch: 24/250 Loss: 6.67443e+05
Epoch: 25/250 Loss: 6.34277e+05
Epoch: 26/250 Loss: 6.02035e+05
Epoch: 27/250 Loss: 5.70717e+05
Epoch: 28/250 Loss: 5.40317e+05
Epoch: 29/250 Loss: 5.10835e+05
Epoch: 30/250 Loss: 4.82268e+05
Epoch: 31/250 Loss: 4.54614e+05
Epoch: 3

In [16]:
params=list(thirddeg.parameters())
print(params)
#print(inputs.shape)

for i, (x,y) in  enumerate(cubic_data):
  x = Variable(x, requires_grad=False)
  y_pred = thirddeg(x)
  print(i,x,y,y_pred)
  

[Parameter containing:
tensor([[1.1556, 1.4950, 3.4785]], requires_grad=True), Parameter containing:
tensor([4.9067], requires_grad=True)]
0 tensor([1.7042]) tensor([28.1368]) tensor([[28.4339]], grad_fn=<AddmmBackward0>)
1 tensor([-0.0796]) tensor([2.2163]) tensor([[4.8224]], grad_fn=<AddmmBackward0>)
2 tensor([-0.2112]) tensor([4.1348]) tensor([[4.6965]], grad_fn=<AddmmBackward0>)
3 tensor([-2.2125]) tensor([-28.6056]) tensor([[-28.0052]], grad_fn=<AddmmBackward0>)
4 tensor([2.3183]) tensor([57.7178]) tensor([[58.9656]], grad_fn=<AddmmBackward0>)
5 tensor([1.0473]) tensor([13.6140]) tensor([[11.7533]], grad_fn=<AddmmBackward0>)
6 tensor([2.0110]) tensor([41.2428]) tensor([[41.5646]], grad_fn=<AddmmBackward0>)
7 tensor([0.9389]) tensor([11.9982]) tensor([[10.1890]], grad_fn=<AddmmBackward0>)
8 tensor([2.9035]) tensor([107.9611]) tensor([[106.0113]], grad_fn=<AddmmBackward0>)
9 tensor([2.8787]) tensor([102.5050]) tensor([[103.6086]], grad_fn=<AddmmBackward0>)
10 tensor([-1.4991]) tenso